# Baseline
* Learns a user vector $u$ and an item vector $a$ and predicts $u_i + a_j$
* Minimizes $L = w_{ij} * (r_{ij} - u_i - a_j) ^ 2 + λ_u * (u_i - μ_u) ^ 2  + λ_a * (a_j - μ_a) ^ 2 $
* This can be computed efficiently via Alternating Least Squares

In [ ]:
medium = ""

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlpha.ipynb");

In [ ]:
import NNlib: sigmoid
import ProgressMeter: @showprogress
import Statistics: mean

In [ ]:
const name = "$medium/Baseline/rating"

In [ ]:
function model(rd::RatingsDataset, userids::Set{Int32})
    df = as_metric(rd, "rating")
    params = read_params(name)
    μ_a, λ_u, λ_a, λ_wu, λ_wa, λ_wt = params["λ"]
    λ_u, λ_a = exp.((λ_u, λ_a))
    λ_wt = sigmoid(λ_wt)
    a = params["a"]
    item_counts = params["a_counts"]

    user_bias = Dict{Int32,Float32}(x => 0 for x in userids)
    denom = Dict{Int32,Float32}(x => λ_u for x in userids)
    @showprogress for i = 1:length(df.userid)
        w = (item_counts[df.itemid[i]]^λ_wa) * (λ_wt^(1 - df.updated_at[i]))
        user_bias[df.userid[i]] += (df.rating[i] - a[df.itemid[i]]) * w
        denom[df.userid[i]] += w
    end
    for k in keys(user_bias)
        user_bias[k] /= denom[k]
    end
    preds = Dict{Int32,Vector{Float32}}()
    @showprogress for u in userids
        preds[u] = a .+ user_bias[u]
    end
    preds
end;

In [ ]:
write_alpha(model, medium, name)
print_losses(medium, "rating", [name]);